In [3]:
import itertools
import numpy as np
import pandas as pd
# for data scaling and splitting
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
# for neural net
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# for evaluation
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
data = pd.read_csv("data/combined_expression.csv")
data['classification'].map({1: 0, 2: 1})
data.head()
data.shape

(642, 16383)

In [5]:
selected_genes = pd.read_csv('cleaned/boruta-99-25-0.01.csv')
selected_genes = selected_genes.values.tolist()
selected_genes = list(itertools.chain(*selected_genes))

In [6]:
# retrieving proper columns
X = data.loc[:, selected_genes]
y = data['classification'].values

# scaling the data
scalar = MinMaxScaler()
x_scaled = scalar.fit_transform(X)

# splitting data (20% test, 80% train)
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2)

# 4 Hidden Layers (with batch normalization)

In [45]:
from tensorflow.keras.layers import BatchNormalization

def create_model(optimizer='rmsprop',init='glorot_uniform', dropout=0.3):
    model = Sequential()
    # adding layers and adding droplayers to avoid overfitting
    hidden_layers = len(selected_genes)
    model.add(Dense(hidden_layers, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))

    model.add(Dense((hidden_layers*1.5), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers*0.5), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    
    model.add(Dense(1, activation='sigmoid'))
    # compiling
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [46]:
# parameters selected from previous gridsearch
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, optimizer='Adam',init='normal')
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Baseline Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Train on 342 samples
Epoch 1/100
342/342 [==============================] - 3s 7ms/sample - loss: 0.8507 - accuracy: 0.5760
Epoch 2/100
342/342 [==============================] - 0s 563us/sample - loss: 0.7010 - accuracy: 0.6608
Epoch 3/100
342/342 [==============================] - 0s 555us/sample - loss: 0.5926 - accuracy: 0.7251
Epoch 4/100
342/342 [==============================] - 0s 567us/sample - loss: 0.5375 - accuracy: 0.7310
Epoch 5/100
342/342 [==============================] - 0s 564us/sample - loss: 0.4646 - accuracy: 0.7719
Epoch 6/100
342/342 [==============================] - 0s 576us/sample - loss: 0.4372 - accuracy: 0.7836
Epoch 7/100
342/342 [==============================] - 0s 576us/sample - loss: 0.3888 - accuracy: 0.8333
Epoch 8/100
342/342 [==============================] - 0s 575us/sample - loss: 0.3754 - accuracy: 0.8216
Epoch 9/100
342/342 [==============================] - 0s 578us/sample - loss: 0.3349 - accuracy: 0.8596
Epoch 10/100
342/342 [==============

342/342 [==============================] - 0s 618us/sample - loss: 0.0185 - accuracy: 0.9942
Epoch 79/100
342/342 [==============================] - 0s 612us/sample - loss: 0.0530 - accuracy: 0.9854
Epoch 80/100
342/342 [==============================] - 0s 638us/sample - loss: 0.0193 - accuracy: 0.9942
Epoch 81/100
342/342 [==============================] - 0s 620us/sample - loss: 0.0204 - accuracy: 0.9912
Epoch 82/100
342/342 [==============================] - 0s 621us/sample - loss: 0.0192 - accuracy: 0.9942
Epoch 83/100
342/342 [==============================] - 0s 624us/sample - loss: 0.0225 - accuracy: 0.9912
Epoch 84/100
342/342 [==============================] - 0s 622us/sample - loss: 0.0139 - accuracy: 0.9971
Epoch 85/100
342/342 [==============================] - 0s 647us/sample - loss: 0.0216 - accuracy: 0.9912
Epoch 86/100
342/342 [==============================] - 0s 634us/sample - loss: 0.0105 - accuracy: 1.0000
Epoch 87/100
342/342 [==============================] - 0s 

342/342 [==============================] - 0s 617us/sample - loss: 0.3886 - accuracy: 0.8187
Epoch 7/100
342/342 [==============================] - 0s 622us/sample - loss: 0.4617 - accuracy: 0.7924
Epoch 8/100
342/342 [==============================] - 0s 630us/sample - loss: 0.3803 - accuracy: 0.8216
Epoch 9/100
342/342 [==============================] - 0s 611us/sample - loss: 0.3248 - accuracy: 0.8567
Epoch 10/100
342/342 [==============================] - 0s 626us/sample - loss: 0.3536 - accuracy: 0.8509
Epoch 11/100
342/342 [==============================] - 0s 613us/sample - loss: 0.3156 - accuracy: 0.8772
Epoch 12/100
342/342 [==============================] - 0s 621us/sample - loss: 0.2931 - accuracy: 0.8889
Epoch 13/100
342/342 [==============================] - 0s 623us/sample - loss: 0.2616 - accuracy: 0.8713
Epoch 14/100
342/342 [==============================] - 0s 627us/sample - loss: 0.2449 - accuracy: 0.9035
Epoch 15/100
342/342 [==============================] - 0s 623

342/342 [==============================] - 0s 674us/sample - loss: 0.0195 - accuracy: 0.9883
Epoch 84/100
342/342 [==============================] - 0s 674us/sample - loss: 0.0141 - accuracy: 0.9971
Epoch 85/100
342/342 [==============================] - 0s 670us/sample - loss: 0.0225 - accuracy: 0.9912
Epoch 86/100
342/342 [==============================] - 0s 656us/sample - loss: 0.0335 - accuracy: 0.9942
Epoch 87/100
342/342 [==============================] - 0s 650us/sample - loss: 0.0173 - accuracy: 0.9912
Epoch 88/100
342/342 [==============================] - 0s 649us/sample - loss: 0.0198 - accuracy: 0.9912
Epoch 89/100
342/342 [==============================] - 0s 646us/sample - loss: 0.0188 - accuracy: 0.9942
Epoch 90/100
342/342 [==============================] - 0s 645us/sample - loss: 0.0159 - accuracy: 0.9971
Epoch 91/100
342/342 [==============================] - 0s 645us/sample - loss: 0.0178 - accuracy: 0.9942
Epoch 92/100
342/342 [==============================] - 0s 

342/342 [==============================] - 0s 602us/sample - loss: 0.3358 - accuracy: 0.8626
Epoch 12/100
342/342 [==============================] - 0s 600us/sample - loss: 0.2877 - accuracy: 0.8860
Epoch 13/100
342/342 [==============================] - 0s 601us/sample - loss: 0.3122 - accuracy: 0.8655
Epoch 14/100
342/342 [==============================] - 0s 603us/sample - loss: 0.1997 - accuracy: 0.9327
Epoch 15/100
342/342 [==============================] - 0s 601us/sample - loss: 0.2778 - accuracy: 0.8918
Epoch 16/100
342/342 [==============================] - 0s 607us/sample - loss: 0.1993 - accuracy: 0.9211
Epoch 17/100
342/342 [==============================] - 0s 601us/sample - loss: 0.1884 - accuracy: 0.9298
Epoch 18/100
342/342 [==============================] - 0s 604us/sample - loss: 0.1751 - accuracy: 0.9211
Epoch 19/100
342/342 [==============================] - 0s 606us/sample - loss: 0.1277 - accuracy: 0.9503
Epoch 20/100
342/342 [==============================] - 0s 

342/342 [==============================] - 0s 660us/sample - loss: 0.0455 - accuracy: 0.9766
Epoch 89/100
342/342 [==============================] - 0s 653us/sample - loss: 0.0352 - accuracy: 0.9825
Epoch 90/100
342/342 [==============================] - 0s 654us/sample - loss: 0.0208 - accuracy: 0.9942
Epoch 91/100
342/342 [==============================] - 0s 673us/sample - loss: 0.0374 - accuracy: 0.9854
Epoch 92/100
342/342 [==============================] - 0s 673us/sample - loss: 0.0277 - accuracy: 0.9912
Epoch 93/100
342/342 [==============================] - 0s 687us/sample - loss: 0.0211 - accuracy: 0.9942
Epoch 94/100
342/342 [==============================] - 0s 668us/sample - loss: 0.0403 - accuracy: 0.9795
Epoch 95/100
342/342 [==============================] - 0s 657us/sample - loss: 0.0347 - accuracy: 0.9883
Epoch 96/100
342/342 [==============================] - 0s 651us/sample - loss: 0.0260 - accuracy: 0.9883
Epoch 97/100
342/342 [==============================] - 0s 

In [47]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Train on 513 samples
Epoch 1/100
513/513 [==============================] - 3s 6ms/sample - loss: 0.8795 - accuracy: 0.5770
Epoch 2/100
513/513 [==============================] - 0s 666us/sample - loss: 0.6639 - accuracy: 0.6784
Epoch 3/100
513/513 [==============================] - 0s 618us/sample - loss: 0.6246 - accuracy: 0.7290
Epoch 4/100
513/513 [==============================] - 0s 627us/sample - loss: 0.5913 - accuracy: 0.7349
Epoch 5/100
513/513 [==============================] - 0s 611us/sample - loss: 0.5120 - accuracy: 0.7446
Epoch 6/100
513/513 [==============================] - 0s 620us/sample - loss: 0.5309 - accuracy: 0.7583
Epoch 7/100
513/513 [==============================] - 0s 628us/sample - loss: 0.4695 - accuracy: 0.7875
Epoch 8/100
513/513 [==============================] - 0s 613us/sample - loss: 0.4490 - accuracy: 0.8070
Epoch 9/100
513/513 [==============================] - 0s 614us/sample - loss: 0.4112 - accuracy: 0.8090
Epoch 10/100
513/513 [==============

513/513 [==============================] - 0s 673us/sample - loss: 0.0565 - accuracy: 0.9805
Epoch 79/100
513/513 [==============================] - 0s 673us/sample - loss: 0.0607 - accuracy: 0.9747
Epoch 80/100
513/513 [==============================] - 0s 673us/sample - loss: 0.0604 - accuracy: 0.9727
Epoch 81/100
513/513 [==============================] - 0s 671us/sample - loss: 0.0250 - accuracy: 0.9922
Epoch 82/100
513/513 [==============================] - 0s 673us/sample - loss: 0.0292 - accuracy: 0.9922
Epoch 83/100
513/513 [==============================] - 0s 668us/sample - loss: 0.0516 - accuracy: 0.9883
Epoch 84/100
513/513 [==============================] - 0s 682us/sample - loss: 0.0339 - accuracy: 0.9883
Epoch 85/100
513/513 [==============================] - 0s 674us/sample - loss: 0.0481 - accuracy: 0.9786
Epoch 86/100
513/513 [==============================] - 0s 671us/sample - loss: 0.0245 - accuracy: 0.9883
Epoch 87/100
513/513 [==============================] - 0s 

In [48]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.76      0.80      0.78        75
           2       0.70      0.65      0.67        54

    accuracy                           0.74       129
   macro avg       0.73      0.72      0.73       129
weighted avg       0.73      0.74      0.73       129



In [50]:
print(confusion_matrix(y_test, y_pred))

[[60 15]
 [19 35]]


# 3 Hidden Layers

In [44]:
def create_model(optimizer='rmsprop',init='glorot_uniform', dropout=0.3):
    model = Sequential()
    # adding layers and adding droplayers to avoid overfitting
    hidden_layers = len(selected_genes)
    model.add(Dense(hidden_layers, activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers*0.5), activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers*0.25), activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1, activation='sigmoid'))
    # compiling
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [65]:
# parameters selected from previous gridsearch
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=32, optimizer='Adam',init='normal')
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Baseline Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Train on 342 samples
Epoch 1/150
342/342 [==============================] - 2s 5ms/sample - loss: 0.6688 - accuracy: 0.6140
Epoch 2/150
342/342 [==============================] - 0s 199us/sample - loss: 0.6034 - accuracy: 0.6637
Epoch 3/150
342/342 [==============================] - 0s 194us/sample - loss: 0.5828 - accuracy: 0.6784
Epoch 4/150
342/342 [==============================] - 0s 202us/sample - loss: 0.5784 - accuracy: 0.6784
Epoch 5/150
342/342 [==============================] - 0s 191us/sample - loss: 0.5433 - accuracy: 0.7251
Epoch 6/150
342/342 [==============================] - 0s 198us/sample - loss: 0.5452 - accuracy: 0.7339
Epoch 7/150
342/342 [==============================] - 0s 196us/sample - loss: 0.5207 - accuracy: 0.7602
Epoch 8/150
342/342 [==============================] - 0s 193us/sample - loss: 0.4762 - accuracy: 0.7719
Epoch 9/150
342/342 [==============================] - 0s 198us/sample - loss: 0.4855 - accuracy: 0.7602
Epoch 10/150
342/342 [==============

Train on 342 samples
Epoch 1/150
342/342 [==============================] - 1s 3ms/sample - loss: 0.6352 - accuracy: 0.6725
Epoch 2/150
342/342 [==============================] - 0s 212us/sample - loss: 0.6197 - accuracy: 0.6667
Epoch 3/150
342/342 [==============================] - 0s 205us/sample - loss: 0.6097 - accuracy: 0.6871
Epoch 4/150
342/342 [==============================] - 0s 195us/sample - loss: 0.5896 - accuracy: 0.7047
Epoch 5/150
342/342 [==============================] - 0s 191us/sample - loss: 0.5662 - accuracy: 0.7222
Epoch 6/150
342/342 [==============================] - 0s 196us/sample - loss: 0.5560 - accuracy: 0.7105
Epoch 7/150
342/342 [==============================] - 0s 202us/sample - loss: 0.5611 - accuracy: 0.7018
Epoch 8/150
342/342 [==============================] - 0s 197us/sample - loss: 0.5377 - accuracy: 0.7368
Epoch 9/150
342/342 [==============================] - 0s 197us/sample - loss: 0.5342 - accuracy: 0.7310
Epoch 10/150
342/342 [==============

Train on 342 samples
Epoch 1/150
342/342 [==============================] - 1s 3ms/sample - loss: 0.6817 - accuracy: 0.5965
Epoch 2/150
342/342 [==============================] - 0s 191us/sample - loss: 0.6514 - accuracy: 0.6199
Epoch 3/150
342/342 [==============================] - 0s 200us/sample - loss: 0.6344 - accuracy: 0.6433
Epoch 4/150
342/342 [==============================] - 0s 202us/sample - loss: 0.6315 - accuracy: 0.6462
Epoch 5/150
342/342 [==============================] - 0s 206us/sample - loss: 0.5995 - accuracy: 0.6667
Epoch 6/150
342/342 [==============================] - 0s 211us/sample - loss: 0.6044 - accuracy: 0.6842
Epoch 7/150
342/342 [==============================] - 0s 211us/sample - loss: 0.6145 - accuracy: 0.6725
Epoch 8/150
342/342 [==============================] - 0s 215us/sample - loss: 0.5822 - accuracy: 0.6959
Epoch 9/150
342/342 [==============================] - 0s 215us/sample - loss: 0.5842 - accuracy: 0.7047
Epoch 10/150
342/342 [==============

Baseline Accuracy: 69.01% (3.60%)


In [42]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Train on 513 samples
Epoch 1/100
513/513 [==============================] - 3s 6ms/sample - loss: 0.8267 - accuracy: 0.5575
Epoch 2/100
513/513 [==============================] - 0s 597us/sample - loss: 0.6675 - accuracy: 0.6881
Epoch 3/100
513/513 [==============================] - 0s 599us/sample - loss: 0.6488 - accuracy: 0.6881
Epoch 4/100
513/513 [==============================] - 0s 600us/sample - loss: 0.6218 - accuracy: 0.7096
Epoch 5/100
513/513 [==============================] - 0s 602us/sample - loss: 0.5363 - accuracy: 0.7427
Epoch 6/100
513/513 [==============================] - 0s 600us/sample - loss: 0.5221 - accuracy: 0.7485
Epoch 7/100
513/513 [==============================] - 0s 603us/sample - loss: 0.4128 - accuracy: 0.8187
Epoch 8/100
513/513 [==============================] - 0s 605us/sample - loss: 0.4357 - accuracy: 0.7914
Epoch 9/100
513/513 [==============================] - 0s 605us/sample - loss: 0.4062 - accuracy: 0.8382
Epoch 10/100
513/513 [==============

513/513 [==============================] - 0s 690us/sample - loss: 0.0427 - accuracy: 0.9844
Epoch 79/100
513/513 [==============================] - 0s 700us/sample - loss: 0.0541 - accuracy: 0.9805
Epoch 80/100
513/513 [==============================] - 0s 714us/sample - loss: 0.0366 - accuracy: 0.9883
Epoch 81/100
513/513 [==============================] - 0s 695us/sample - loss: 0.0549 - accuracy: 0.9786
Epoch 82/100
513/513 [==============================] - 0s 703us/sample - loss: 0.0606 - accuracy: 0.9805
Epoch 83/100
513/513 [==============================] - 0s 695us/sample - loss: 0.0681 - accuracy: 0.9727
Epoch 84/100
513/513 [==============================] - 0s 698us/sample - loss: 0.0340 - accuracy: 0.9864
Epoch 85/100
513/513 [==============================] - 0s 693us/sample - loss: 0.0493 - accuracy: 0.9786
Epoch 86/100
513/513 [==============================] - 0s 693us/sample - loss: 0.0618 - accuracy: 0.9805
Epoch 87/100
513/513 [==============================] - 0s 

In [43]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.68      0.79      0.73        75
           2       0.62      0.48      0.54        54

    accuracy                           0.66       129
   macro avg       0.65      0.63      0.64       129
weighted avg       0.65      0.66      0.65       129



In [63]:
print(confusion_matrix(y_test, y_pred))

[[75  0]
 [54  0]]


# 2 Hidden Layers

In [15]:
def create_model(optimizer='rmsprop',init='glorot_uniform', dropout=0.3):
    model = Sequential()
    # adding layers and adding droplayers to avoid overfitting
    hidden_layers = len(selected_genes)
    model.add(Dense(hidden_layers, activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers*0.5), activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1, activation='sigmoid'))
    # compiling
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [16]:
# parameters selected from previous gridsearch
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=64, optimizer='Adagrad',init='uniform')
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Baseline Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 3ms/sample - loss: 0.6388 - accuracy: 0.6637
Epoch 2/50
342/342 [==============================] - 0s 123us/sample - loss: 0.6100 - accuracy: 0.6842
Epoch 3/50
342/342 [==============================] - 0s 123us/sample - loss: 0.5860 - accuracy: 0.7018
Epoch 4/50
342/342 [==============================] - 0s 125us/sample - loss: 0.5731 - accuracy: 0.6988
Epoch 5/50
342/342 [==============================] - 0s 128us/sample - loss: 0.5647 - accuracy: 0.7281
Epoch 6/50
342/342 [==============================] - 0s 127us/sample - loss: 0.5718 - accuracy: 0.6988
Epoch 7/50
342/342 [==============================] - 0s 126us/sample - loss: 0.5551 - accuracy: 0.7310
Epoch 8/50
342/342 [==============================] - 0s 129us/sample - loss: 0.5326 - accuracy: 0.7310
Epoch 9/50
342/342 [==============================] - 0s 127us/sample - loss: 0.5445 - accuracy: 0.7485
Epoch 10/50
342/342 [========================

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 3ms/sample - loss: 0.6668 - accuracy: 0.6257
Epoch 2/50
342/342 [==============================] - 0s 122us/sample - loss: 0.6618 - accuracy: 0.5936
Epoch 3/50
342/342 [==============================] - 0s 122us/sample - loss: 0.6375 - accuracy: 0.6491
Epoch 4/50
342/342 [==============================] - 0s 127us/sample - loss: 0.6278 - accuracy: 0.6550
Epoch 5/50
342/342 [==============================] - 0s 130us/sample - loss: 0.6253 - accuracy: 0.6345
Epoch 6/50
342/342 [==============================] - 0s 126us/sample - loss: 0.6254 - accuracy: 0.6287
Epoch 7/50
342/342 [==============================] - 0s 127us/sample - loss: 0.6178 - accuracy: 0.6345
Epoch 8/50
342/342 [==============================] - 0s 126us/sample - loss: 0.6076 - accuracy: 0.6725
Epoch 9/50
342/342 [==============================] - 0s 124us/sample - loss: 0.6181 - accuracy: 0.6696
Epoch 10/50
342/342 [========================

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 3ms/sample - loss: 0.6620 - accuracy: 0.6257
Epoch 2/50
342/342 [==============================] - 0s 126us/sample - loss: 0.6110 - accuracy: 0.6667
Epoch 3/50
342/342 [==============================] - 0s 129us/sample - loss: 0.6040 - accuracy: 0.6667
Epoch 4/50
342/342 [==============================] - 0s 131us/sample - loss: 0.5976 - accuracy: 0.6813
Epoch 5/50
342/342 [==============================] - 0s 131us/sample - loss: 0.5781 - accuracy: 0.6784
Epoch 6/50
342/342 [==============================] - 0s 132us/sample - loss: 0.5615 - accuracy: 0.7018
Epoch 7/50
342/342 [==============================] - 0s 137us/sample - loss: 0.5734 - accuracy: 0.6988
Epoch 8/50
342/342 [==============================] - 0s 136us/sample - loss: 0.5638 - accuracy: 0.7047
Epoch 9/50
342/342 [==============================] - 0s 139us/sample - loss: 0.5589 - accuracy: 0.7222
Epoch 10/50
342/342 [========================

Baseline Accuracy: 74.27% (7.21%)


In [68]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Train on 513 samples
Epoch 1/150
513/513 [==============================] - 1s 2ms/sample - loss: 0.6178 - accuracy: 0.6413
Epoch 2/150
513/513 [==============================] - 0s 204us/sample - loss: 0.6409 - accuracy: 0.6316
Epoch 3/150
513/513 [==============================] - 0s 204us/sample - loss: 0.6027 - accuracy: 0.6686
Epoch 4/150
513/513 [==============================] - 0s 208us/sample - loss: 0.6361 - accuracy: 0.6433
Epoch 5/150
513/513 [==============================] - 0s 221us/sample - loss: 0.5810 - accuracy: 0.7057
Epoch 6/150
513/513 [==============================] - 0s 223us/sample - loss: 0.5777 - accuracy: 0.7018
Epoch 7/150
513/513 [==============================] - 0s 211us/sample - loss: 0.6281 - accuracy: 0.6569
Epoch 8/150
513/513 [==============================] - 0s 205us/sample - loss: 0.5792 - accuracy: 0.6901
Epoch 9/150
513/513 [==============================] - 0s 207us/sample - loss: 0.5540 - accuracy: 0.6901
Epoch 10/150
513/513 [==============

In [69]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.68      0.87      0.76        75
           2       0.71      0.44      0.55        54

    accuracy                           0.69       129
   macro avg       0.70      0.66      0.66       129
weighted avg       0.69      0.69      0.67       129



In [19]:
print(confusion_matrix(y_test, y_pred))

[[68  7]
 [33 21]]


In [20]:
model.model.save('models/hidden_2.h5')

# 1 Hidden Layer

In [21]:
def create_model(optimizer='rmsprop',init='glorot_uniform', dropout=0.3):
    model = Sequential()
    # adding layers and adding droplayers to avoid overfitting
    hidden_layers = len(selected_genes)
    model.add(Dense(hidden_layers, activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1, activation='sigmoid'))
    # compiling
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [23]:
# parameters selected from previous gridsearch
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=64, optimizer='Adagrad',init='uniform')
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Baseline Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 2ms/sample - loss: 0.7090 - accuracy: 0.5614
Epoch 2/50
342/342 [==============================] - 0s 98us/sample - loss: 0.6379 - accuracy: 0.6579
Epoch 3/50
342/342 [==============================] - 0s 98us/sample - loss: 0.6177 - accuracy: 0.6667
Epoch 4/50
342/342 [==============================] - 0s 97us/sample - loss: 0.6061 - accuracy: 0.6608
Epoch 5/50
342/342 [==============================] - 0s 102us/sample - loss: 0.5950 - accuracy: 0.6725
Epoch 6/50
342/342 [==============================] - 0s 99us/sample - loss: 0.6000 - accuracy: 0.6754
Epoch 7/50
342/342 [==============================] - 0s 100us/sample - loss: 0.5826 - accuracy: 0.7105
Epoch 8/50
342/342 [==============================] - 0s 100us/sample - loss: 0.5896 - accuracy: 0.6813
Epoch 9/50
342/342 [==============================] - 0s 97us/sample - loss: 0.5772 - accuracy: 0.6813
Epoch 10/50
342/342 [=============================

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 2ms/sample - loss: 0.6836 - accuracy: 0.6023
Epoch 2/50
342/342 [==============================] - 0s 97us/sample - loss: 0.6214 - accuracy: 0.6784
Epoch 3/50
342/342 [==============================] - 0s 101us/sample - loss: 0.6072 - accuracy: 0.6871
Epoch 4/50
342/342 [==============================] - 0s 103us/sample - loss: 0.6039 - accuracy: 0.6842
Epoch 5/50
342/342 [==============================] - 0s 104us/sample - loss: 0.5949 - accuracy: 0.6813
Epoch 6/50
342/342 [==============================] - 0s 103us/sample - loss: 0.5951 - accuracy: 0.6842
Epoch 7/50
342/342 [==============================] - 0s 99us/sample - loss: 0.5769 - accuracy: 0.6842
Epoch 8/50
342/342 [==============================] - 0s 102us/sample - loss: 0.5672 - accuracy: 0.7251
Epoch 9/50
342/342 [==============================] - 0s 103us/sample - loss: 0.5640 - accuracy: 0.6988
Epoch 10/50
342/342 [==========================

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 3ms/sample - loss: 0.6816 - accuracy: 0.5702
Epoch 2/50
342/342 [==============================] - 0s 96us/sample - loss: 0.6308 - accuracy: 0.6520
Epoch 3/50
342/342 [==============================] - 0s 102us/sample - loss: 0.6148 - accuracy: 0.6462
Epoch 4/50
342/342 [==============================] - 0s 100us/sample - loss: 0.6082 - accuracy: 0.6901
Epoch 5/50
342/342 [==============================] - 0s 105us/sample - loss: 0.6147 - accuracy: 0.6637
Epoch 6/50
342/342 [==============================] - 0s 107us/sample - loss: 0.5963 - accuracy: 0.6754
Epoch 7/50
342/342 [==============================] - 0s 104us/sample - loss: 0.5871 - accuracy: 0.6813
Epoch 8/50
342/342 [==============================] - 0s 103us/sample - loss: 0.5919 - accuracy: 0.6930
Epoch 9/50
342/342 [==============================] - 0s 104us/sample - loss: 0.5736 - accuracy: 0.7018
Epoch 10/50
342/342 [=========================

Baseline Accuracy: 71.54% (1.81%)


In [24]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Train on 513 samples
Epoch 1/50
513/513 [==============================] - 1s 1ms/sample - loss: 0.6378 - accuracy: 0.6550
Epoch 2/50
513/513 [==============================] - 0s 108us/sample - loss: 0.6064 - accuracy: 0.6764
Epoch 3/50
513/513 [==============================] - 0s 109us/sample - loss: 0.6049 - accuracy: 0.6803
Epoch 4/50
513/513 [==============================] - 0s 111us/sample - loss: 0.5766 - accuracy: 0.6940
Epoch 5/50
513/513 [==============================] - 0s 103us/sample - loss: 0.5789 - accuracy: 0.7037
Epoch 6/50
513/513 [==============================] - 0s 104us/sample - loss: 0.5901 - accuracy: 0.6803
Epoch 7/50
513/513 [==============================] - 0s 103us/sample - loss: 0.5871 - accuracy: 0.7251
Epoch 8/50
513/513 [==============================] - 0s 105us/sample - loss: 0.5732 - accuracy: 0.6940
Epoch 9/50
513/513 [==============================] - 0s 106us/sample - loss: 0.5794 - accuracy: 0.6979
Epoch 10/50
513/513 [========================

In [25]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.67      0.93      0.78        75
           2       0.79      0.35      0.49        54

    accuracy                           0.69       129
   macro avg       0.73      0.64      0.63       129
weighted avg       0.72      0.69      0.66       129



In [26]:
print(confusion_matrix(y_test, y_pred))

[[70  5]
 [35 19]]


In [27]:
model.model.save('models/hidden_1.h5')